In [1]:
#=====================================================
#City Subway Network Optimization - Data Reading
#=====================================================

#Reading data downloaded from the US Census website for all census blocks in 
cityCensusBlockPopulations = open("ColumbusCensusBlockPopulations.csv", 'r')

#rows is the entire csv in one string
rows = cityCensusBlockPopulations.read().split("\n")
headers = rows[0].split(", Franklin County, Ohio")

#the last value is empty so we can get rid of it to make the next list creation easier (avoid an index searching error)
headers.pop()

#Creating lists that hold the tract and block numbers for the city metro area census data
cityTracts = []
cityBlocks = []
for label in headers:
    cityBlocks.append(label[label.index("Block") + 6:label.index("Block") + 10])
    cityTracts.append(label[label.index("Tract") + 6:])
    
#Creating the list that holds the population in each block from the city data
cityBlockPopulations = rows[1].split(",")

#removing the first value twice because the first two just say "Total" and then the value for the entire metro area which is not needed
cityBlockPopulations.pop(0)
cityBlockPopulations.pop(0)
    
cityCensusBlockPopulations.close()

#Creating one list of unique tract-block identifiers
city_ids = []
for index in range(len(cityBlocks)):
    
    tract = cityTracts[index]
    block = cityBlocks[index]    
    
    #some tracts are represented in this dataset as single numbers - they should have two trailing zeros, which are the numbers that come after the decimal point
    if "." not in tract:
        tract = tract + "00"
    else: #remove the decimal
        decimal_index = tract.find(".")
        tract = tract[:decimal_index] + tract[decimal_index + 1:]
        
    #reformatting to add leadering zeroes when necessary for consistent formatting with the other dataset
    while len(tract) < 6:
        tract = "0" + tract
    
    unique_id = tract + "-" + block
    city_ids.append(unique_id)
    
#================================================================================================================================    

#Reading in the geodata for the entire state of ohio
stateGeodata = open("OhioGeodata.csv", 'r')

#rows is the entire csv in one string
rows = stateGeodata.read().split("\n")
rows.pop() #the last row is just empty and indexing later will be an issue if we leave it there
headers = rows[0].split(",")
blockTypeIndex = headers.index('"Land/Water Block Type"')
idIndex = headers.index('"Geographic Identifier"')
internal_point_latitude_index = headers.index('"Internal Point Latitude"')
internal_point_longitude_index = headers.index('"Internal Point Longitude"')

#some blocks have the same tract/block ID but are a different block type (water, land, both). We want land when possible, and both as a backup.
#Water-only blocks have no population and are therefore not needed
land_and_water_ids = []
state_ids = []

state_latitudes = []
land_and_water_latitudes = []

state_longitudes = []
land_and_water_longitudes = []

#getting the tract/block ids from the state geodata
for rowNum in range(1, len(rows)):
    rowData = rows[rowNum].split(",")
    
    unique_id = rowData[idIndex][-11:-5] + "-" + rowData[idIndex][-5:-1]
    latitude = rowData[internal_point_latitude_index]
    longitude = rowData[internal_point_longitude_index]
    
    #some blocks are entirely on land, some are a mix of land and water. We want the land ones by default, and the land+water ones when there is no land one by the same id
    if rowData[blockTypeIndex] == '"L"':
        state_ids.append(unique_id)
        state_latitudes.append(latitude)
        state_longitudes.append(longitude)
        
    elif rowData[blockTypeIndex] == '"B"':
        land_and_water_ids.append(unique_id)
        land_and_water_latitudes.append(latitude)
        land_and_water_longitudes.append(longitude)
        
#adding the mixed land/water ids and coordinates when they aren't already present, but defaulting to the land ones when they exist
for identifier_index in range(len(land_and_water_ids)):
    
    if land_and_water_ids[identifier_index] not in state_ids:
        state_ids.append(land_and_water_ids[identifier_index])
        state_latitudes.append(land_and_water_latitudes[identifier_index])
        state_longitudes.append(land_and_water_longitudes[identifier_index])
        
#assembling matching lists:
#ids: a list of unique ids (T is a tract number, B is a block number) in form TTTTTT-BBBB, all must be shared between the city and state data
#poulations: a list of the populations for all unique blocks in the same order as ids
#longitudes: a list of the longitudes of the census block internal points
#latitudes: a list of the latitudes of the census block internal points

ids = []
populations = []
latitudes = []
longitudes = []

for id_index in range(len(state_ids)):
    if state_ids[id_index] in city_ids and state_ids[id_index] not in ids:
        ids.append(state_ids[id_index])
        latitudes.append(float(state_latitudes[id_index]))
        longitudes.append(float(state_longitudes[id_index]))
        
        popIndex = city_ids.index(state_ids[id_index])
        populations.append(int(float(cityBlockPopulations[popIndex])))
    
stateGeodata.close()

In [ ]:
#import packages
import gurobipy as gp
from gurobipy import GRB

In [45]:
#=====================================================
#City Subway Network Optimization - Point Selection
#=====================================================

#model creation
point_selection_mod = gp.Model("City Subway Point Selection")

#creating the list of point objects
points = []

#creating the point objects
for blockIndex in range(len(ids)):
    points.append(point_selection_mod.addVar(obj = populations[blockIndex], vtype = "B", name = ids[blockIndex]))
    
#the model may only distribute 10 stations
point_selection_mod.addConstr(gp.quicksum(points) == 10)

#maximize population reached
point_selection_mod.ModelSense = 0
point_selection_mod.optimize()

#creating the list of points that were selected
points_selected = []
if point_selection_mod.Status == gp.GRB.OPTIMAL:
    print("Population reached =", point_selection_mod.ObjVal)
    for point in points:
        if point.X == 1:
            points_selected.append(point.Varname)
            


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 16181 columns and 16181 nonzeros
Model fingerprint: 0x8026e0f4
Variable types: 0 continuous, 16181 integer (16181 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]
Found heuristic solution: objective 1161.0000000
Presolve removed 1 rows and 16181 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.01 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 17410 1161 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.741000000000e+04, best bound 1.741000000000e+04, gap 0.0000%
Pop

In [42]:
import pandas as pd
import folium
import geopy.distance
from itertools import combinations, permutations

In [46]:
#=====================================================
#City Subway Network Optimization - Station Mapping
#=====================================================

#Creating the pandas dataframe to run the route optimization from
latitudesColumn = []
longitudesColumn = []
for point in points_selected:
    lat_long_index = ids.index(point)
    latitudesColumn.append(latitudes[lat_long_index])
    longitudesColumn.append(longitudes[lat_long_index])
    
stationsDoL = {"Name": points_selected,
               "Latitude": latitudesColumn,
              "Longitude": longitudesColumn}
stations = pd.DataFrame(stationsDoL, index = points_selected)
stations = stations.rename_axis('ID')

In [47]:
#Visual map of the stations
map = folium.Map(location=[40, -95], zoom_start=4)
for idx, station in stations.iterrows():
    folium.Marker([station['Latitude'],station['Longitude']],
                            popup=station['Name'],
                             icon=folium.Icon(color= 'blue')).add_to(map)
map

In [32]:
station_indexes = stations.index

#Function to calculate distance between stations
def distance(station1, station2):
    
    distance = geopy.distance.distance((stations.at[station1,'Latitude'],stations.at[station1,'Longitude']),
                        (stations.at[station2,'Latitude'],stations.at[station2,'Longitude'])).mi  
    return distance

distances = {(station1, station2): distance(station1, station2) for station1, station2 in permutations(station_indexes, 2)}

In [36]:
#Create subtour elimination constraint function (this code chunk originates from Dr. Anthony Bonifonte at Denison University)

## Calculate shortest subtour for subtour elimination constraints
def subtour(edges):
    nodes = set(i for e in edges for i in e)
    unvisited = list(nodes)
    cycle = list(nodes)
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*') if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle # New shortest subtour
    return cycle

## Subtour elimination function
def subtourelim(model, where):
    global subtour_iterations
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys()
            if vals[i, j] > 0.5)
        tour = subtour(selected)
        if len(tour) < len(selected): #len(selected) is total number of edges
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in permutations(tour, 2)) <= len(tour)-1)

In [51]:
#=====================================================
#City Subway Network Optimization - Route Optimization
#=====================================================

station_model = gp.Model("Subway Route Design")

# Variables: is station 'i' adjacent to station 'j' on the route?
vars = station_model.addVars(distances.keys(), obj=distances, vtype=GRB.BINARY, name='Station Pairs')

# Constraints: one incoming and one outgoing edge to each capital
cons1 = station_model.addConstrs(vars.sum(m, '*') == 1 for m in station_indexes)
cons2 = station_model.addConstrs(vars.sum('*',m ) == 1 for m in station_indexes)

# Add all constraints for pairwise violated subtours
station_model.addConstrs(vars[(m1, m2)] + vars[(m2, m1)] <= 1 for m1 in station_indexes for m2 in station_indexes if m1!=m2)

station_model._vars = vars
station_model.Params.lazyConstraints = 1
station_model.optimize(subtourelim)

vals = station_model.getAttr('x', vars)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)
#selected

map = folium.Map(location=[40, -95], zoom_start=4)
for idx, station in stations.iterrows():
    folium.Marker([station['Latitude'],station['Longitude']],
                            popup=station['Name'],
                             icon=folium.Icon(color= 'blue')).add_to(map)
  
for loc in selected:
    folium.PolyLine([[stations.at[loc[0],'Latitude'],stations.at[loc[0],'Longitude']],
                     [stations.at[loc[1],'Latitude'],stations.at[loc[1],'Longitude']]]).add_to(map)
    

map

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 110 rows, 90 columns and 360 nonzeros
Model fingerprint: 0x13e05261
Variable types: 0 continuous, 90 integer (90 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 61.6155597
Presolve removed 45 rows and 0 columns
Presolve time: 0.01s
Presolved: 65 rows, 90 columns, 270 nonzeros
Variable types: 0 continuous, 90 integer (90 binary)

Root relaxation: objective 3.213503e+01, 27 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntI